# Extract census tract averages from model data
In this notebook, I extract the spatial averages from the previous research model outputs, so that I can get the spatial mean for each location.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import rasterio as rio
import geopandas as gpd
import pandas as pd

In [2]:
# Open the block group and census tract level TIFFs
with rio.open('../data/preprocessed/census/census_tracts_by_spatial_model_geoid.tif') as src:
    ct = src.read(1)
    ct_meta = src.meta

with rio.open('../data/preprocessed/census_blocks/block_group_by_spatial_model_geoid.tif') as src:
    bg = src.read(1)
    bg_meta = src.meta

In [3]:
temp_path = '../data/preprocessed/spatial_causal_model_results/durham_temp_celsius.tif'
confound_path = '../data/preprocessed/spatial_causal_model_results/durham_temp_confound.tif'
no_unobs_path = '../data/preprocessed/spatial_causal_model_results/durham_temp_celsius_no_unobs.tif'
confound_dp = '../data/preprocessed/spatial_causal_model_results/durham_temp_confound_dot_product.tif'
# Open the temperature TIFFs
with rio.open(temp_path) as src:
    temp = src.read(1)
    temp_meta = src.meta

with rio.open(confound_path) as src:
    confound = src.read(1)
    confound_meta = src.meta

with rio.open(no_unobs_path) as src:
    no_unobs = src.read(1)
    no_unobs_meta = src.meta

with rio.open(confound_dp) as src:
    confound_dp = src.read(1)
    confound_dp_meta = src.meta

In [4]:
geoids = []
temp_means = []
confound_means = []
confound_dp_means = []
no_unobs_means = []

for tract in np.unique(ct):
    mask = ct == tract
    geoids.append(tract)
    temp_means.append(np.nanmean(temp[mask]))
    confound_means.append(np.nanmean(confound[mask]))
    confound_dp_means.append(np.nanmean(confound_dp[mask]))
    no_unobs_means.append(np.nanmean(no_unobs[mask]))

In [5]:
# Turn into a pandas dataframe
df = pd.DataFrame({'geoid': geoids, 'temp': temp_means, 'confound': confound_means, 'no_unobs': no_unobs_means, 'counfound_dp': confound_dp_means})

df.head()

,geoid,temp,confound,no_unobs,counfound_dp
0,0,26.394690,0.160840,26.253919,0.134647
1,37063000101,26.015177,-0.899160,26.914338,-0.441683
2,37063000102,26.032398,-1.058885,27.091283,-0.336193
3,37063000200,26.700919,-0.411577,27.112496,-0.271472
4,37063000301,26.968267,-0.535571,27.503838,-0.114608


In [6]:
# Save the dataframe
df.to_csv('../data/preprocessed/spatial_causal_model_results/by_census_tract.csv', index=False)

In [7]:
geoids = []
temp_means = []
confound_means = []
confound_dp_means = []
no_unobs_means = []

for group in np.unique(bg):
    mask = bg == group
    geoids.append(group)
    temp_means.append(np.nanmean(temp[mask]))
    confound_means.append(np.nanmean(confound[mask]))
    no_unobs_means.append(np.nanmean(no_unobs[mask]))
    confound_dp_means.append(np.nanmean(confound_dp[mask]))

In [8]:
# Create a dataframe
df = pd.DataFrame({'geoid': geoids, 'temp': temp_means, 'confound': confound_means, 'no_unobs': no_unobs_means, 'counfound_dp': confound_dp_means})
df.head()

,geoid,temp,confound,no_unobs,counfound_dp
0,0,26.394690,0.160840,26.253919,0.134647
1,370630001011,26.242741,-0.813138,27.055879,-0.398611
2,370630001012,25.724601,-1.009002,26.733603,-0.496682
3,370630001021,25.932258,-1.373066,27.305325,-0.277743
4,370630001022,25.946321,-0.746686,26.693007,-0.330941


In [9]:
# save the dataframe
df.to_csv('../data/preprocessed/spatial_causal_model_results/by_block_group.csv', index=False)

In [10]:
# Confirm that all of the bounds are the same...
files = [
    '../data/preprocessed/spatial_causal_model_results/durham_temp_celsius.tif',
    '../data/preprocessed/spatial_causal_model_results/durham_temp_confound.tif',
    '../data/preprocessed/spatial_causal_model_results/durham_temp_celsius_no_unobs.tif',
    '../data/preprocessed/spatial_causal_model_results/durham_temp_confound_dot_product.tif'
]

bounds = []

for fn in files:
    with rio.open(fn) as src:
        bounds.append(src.bounds)


# print(bounds)

# Are all bounds equal?
print(all([b == bounds[0] for b in bounds]))


True
